# Évitement des inter-blocages
## Algorithme de sûreté

In [1]:
import numpy as np
import pandas as pd
import display

def append(df: pd.DataFrame, series: pd.Series) -> pd.DataFrame:
    return pd.concat((df, series.to_frame().T), ignore_index=True)

EXEMPLE = "001"

In [2]:
Allocation = pd.read_csv(f"exemples/evitement-interblocage/{EXEMPLE}/allocation.csv")
Max = pd.read_csv(f"exemples/evitement-interblocage/{EXEMPLE}/max.csv")
Ressources = pd.read_csv(f"exemples/evitement-interblocage/{EXEMPLE}/ressources.csv")

M = Ressources.shape[1]
N = Allocation.shape[0]

display.dataframes({'Allocation': Allocation, 'Max': Max, 'Ressources': Ressources})

,A,B,C
0,0,1,0
1,2,0,0
2,3,0,2
3,2,1,1
4,0,0,2
,A,B,C
0,7,5,3
1,3,2,2
2,9,0,2
3,2,2,2


Déterminer le vecteur _Disponible_ et la matrice _Besoin_ ?

In [3]:
Utilise = pd.DataFrame([Allocation.sum()])
Disponible = Ressources - Utilise

display.dataframes({'Disponible': Disponible})

,A,B,C
0,3,3,2


In [4]:
Besoin = Max - Allocation

display.dataframes({'Besoin': Besoin})

,A,B,C
0,7,4,3
1,1,2,2
2,6,0,0
3,0,1,1
4,4,3,1


### 1. Soit _Travail_ et _Fin_ des tableaux (vecteurs) de longueurs _m_ et _n_, respectivement.

On initialise :
```
Travail = Disponible
Fin[i] = false pour i = 0, 1, ..., n -1
```

In [5]:
Travail = Disponible.copy()

Fin = np.zeros((1, N), dtype=np.bool_)
Fin = pd.DataFrame(Fin)

display.dataframes({'Travail': Travail, 'Fin': Fin})

Travail 
 
 
 
 A 
 B 
 C 
 
 
 
 
 0 
 3 
 3 
 2 
 
 
 Fin 
 
 
 
 0 
 1 
 2 
 3 
 4 
 
 
 
 
 0 
 False 
 False 
 False 
 False 
 False

### 2. Trouve un index _i_ tel que: 

```
(a) Fin [i] = false
(b) Besoin[i] <= Travail
```

Si un tel _i_ n’existe pas aller à étape 4.

In [6]:
Chemin = []

In [17]:
i = None

for j in range(N):
    if not Fin.iloc[-1, j] and (Besoin.iloc[j] <= Travail.iloc[-1]).all():
        i = j
        break

if i is not None:
    Chemin.append(i)

print(f"i: {i}")

i: None


### 3. Mettre à jour _Travail_ et _Fin[i]_:

```
Travail = Travail + Allocation[i]
Fin[i] = true
```

Aller à étape 2.

In [16]:
Travail = append(Travail, Travail.iloc[-1] + Allocation.iloc[i])
Fin = append(Fin, Fin.iloc[-1])

Fin.iloc[-1, i] = True

display.dataframes({'Travail': Travail, 'Fin': Fin})

Travail 
 
 
 
 A 
 B 
 C 
 
 
 
 
 0 
 3 
 3 
 2 
 
 
 1 
 5 
 3 
 2 
 
 
 2 
 7 
 4 
 3 
 
 
 3 
 7 
 5 
 3 
 
 
 4 
 10 
 5 
 5 
 
 
 5 
 10 
 5 
 7 
 
 
 Fin 
 
 
 
 0 
 1 
 2 
 3 
 4 
 
 
 
 
 0 
 False 
 False 
 False 
 False 
 False 
 
 
 1 
 False 
 True 
 False 
 False 
 False 
 
 
 2 
 False 
 True 
 False 
 True 
 False 
 
 
 3 
 True 
 True 
 False 
 True 
 False 
 
 
 4 
 True 
 True 
 True 
 True 
 False 
 
 
 5 
 True 
 True 
 True 
 True 
 True

### 4. Si `Fin[i] == true` pour tout _i_, alors le système est dans un état sûr:

In [18]:
Sur = Fin.iloc[-1].all()

print(f"Sur = {Sur}")

if Sur:
    print(f"Chemin: {', '.join(map(lambda i: f'P{i}', Chemin))}")

Sur = True
Chemin: P1, P3, P0, P2, P4
